교차 검증과 그리드 서치

In [ ]:
# 검증 세트 (Validation set): 테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합하기 어려워서
#   훈련 세트를 또 나누어 만드는 세트 

In [ ]:
# 검증 세트 만들기

import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
wine.columns

In [ ]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)


In [ ]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [ ]:
print(sub_input.shape, val_input.shape)

In [ ]:
# 훈련세트와 검증세트로 모델링

from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input,val_target))

#  과대적합됨. 매개변수를 바꾸서 더 좋은 모델을 찾아야 함.

교차 검증(cross validation)

In [ ]:
# 보통 많은 데이터를 훈련에 사용할 수록 좋은 모델이 나오는데 검증 세트를 만드느라 훈련 세트가 줄었음.
# 그렇다고 검증 세트를 너무 조금 떼어 놓으면 검증 점수가 들쭉날쭉하고 불아정 해짐.

# >> 이럴 때 교차검증 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이어 사용 가능.

In [ ]:
# ** 교차 검증 **

# ** 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복함.
# 그 다음 이 점수를 평균하여 최종 검증 점수를 얻음.


# 3-폴드 교차 검증
# 훈련 세트를 세 부분으로 나눠서 교차 검증을 수행하는 것을 3-폴드 교차 검증이라고 함.
# = k-폴드 교차 검증(k-fold cross validation)이라고 하며, 훈련 세트를 몇 부분으로 나누냐에 따라
# 다르게 부름. (=k 겹 교차 검증)

In [ ]:
# Cross_validate() 교차 검증 함수
# - 평가할 모델 객체를 첫 번째 매개변수로 전달함.
# - 그 다음 앞에서처럼 직접 검증 세트를 떼어 내지 않고 훈련 세트 전체를 cross vaildate()함수에 전달


from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

In [ ]:
# fit_time, score_time, test_sore 키를 가진 딕셔너리로 반환.

# 처음 2개의 키는 각각 모델을 훈련하는 시간과 검증하는 시간.
# 각 키마다 5개의 숫자가 담겨 있음.
# cross_validate함수는 기본적으로 5-폴드 교차 검증을 수행함.
# cv 매개변수에서 폴드 수를 바꿀 수도 있음.

# cross_validate() 함수의 전산인 cross_val_score() 함수도 있음.
# 이 함수는 cross_validate()함수의 결과 중에서 test_score 값만 반환

In [ ]:
# 교차 검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있음.

import numpy as np
print(np.mean(scores['test_score']))

In [ ]:
# 교차 검증을 수행하면 입력한 모델에서 얻을 수 있는 최상의 검증 점수를 가늠할 수 있음!

In [ ]:
# 주의할 점. 

# cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않음.
# train_test_split() 함수로 전체 데이터를 섞은 후 세트를 준비했기 때문에 따로 섞을 필요가 없었지만
# 교차 검증을 할 때 훈련 세트를 섞으려면 분할기(splitter)를 지정해야 함.

# 사이키런의 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정해줌.
# cross_validate() 함수는 기본적으로 회귀 모델일 경우 KFold 분할기를 사용하고
# 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold를 사용함.


from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

In [ ]:
# 훈련 세트를 섞은 후 10- 폴드 교차 검증을 수행하려면

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

하이퍼 파라미터 튜닝

In [ ]:
# 모델 파라미터 : 머신러닝 모델이 학습하는 파라미터

# 하이퍼 파라미터 : 모델이 학습할 수 없어서 사용자가 지정해줘야 하는 파라미터

# 사이킷런과 같은 머신러닝 라이브러리를 사용할 때 
# 하이퍼 파라미터 >> 클래스나 메서드의 매개변수로 표현됨.

In [ ]:
# 하이퍼 파라미터 튜닝 작업
# - 라이브러리가 제공하는 기본값을 그대로 사용해 모델을 훈련
# - 검증 세트의 점수나 교차 검증을 통해서 매개변수를 조금씩 바꿈.
# - 모델마다 적게는 1~2개 많으면 5~6개 매개변수 제공.
# - 이 매개변수를 바꿔가면서 교차 검증 수행.

In [ ]:
# AutoML 사람의 개입 없이 하이퍼 파라미터 튜닝을 자동으로 수행하는 기술

In [ ]:
# 결정 트리 모델에서 최적의 max_depth을 찾고 min_samples_split을 바꿔가며 최적의 값을
# 찾을 수 없음!  max_depth의 최적값은 min_samples_split매개변수의 값이 바뀌면 함께 달라짐.
# 즉 이 두 매개 변수를 동시에 바꿔가며 최적의 값을 찾아야 함.

In [ ]:
# 검증 세트 만들기

import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
wine.columns

In [ ]:
import numpy as np
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

교차 검증

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(train_input, train_target)

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target, cv= StratifiedKFold())
scores


In [ ]:
print(np.mean(scores['test_score']))

In [ ]:
# 훈련세트를 10-폴드 교차 검증 수행


# train, test 세트까지 나눴다고 가정.

from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(train_input, train_target)

from sklearn.model_selection import StratifiedKFold
spliter = StratifiedKFold(n_splits=10, shuffle=True)

from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target, cv = spliter)
scores

In [ ]:
print(np.mean(scores['test_score']))

그리드 서치(Grid Search)

- GridSearchCV클래스
    - 하이퍼 파라미터 탐색과 교차 검증을 한 번에 수행.

In [ ]:
# 기본 매개변수를 사용한 결정 트리 모델에서 min_impurity_decrease 매개변수의 최적값 찾기

# 우선 GridSearchCV 클래스 임포트 후 탐색할 매개변수 및 탐색할 값 리스트 딕셔너리로 만들기

from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001,0.0002,0.0003, 0.0004, 0.0005]}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
# GridSearchCV 의 cv 매개변수 기본값은 5
# min_impurity_decrease 값마다 5번씩 폴드 교차 검증을 수행함. >> 총 25번!
# 많은 모델을 훈련하기 떄문에 n_jobs= 매개변수에 병렬실행에 사용할 cpu 코어 수 지정
# 기본값 1 , -1 지정하면 시스템에 있는 모든 코어 사용

In [ ]:
gs.fit(train_input, train_target)

In [ ]:
# 교차 검증에서 최적의 하이퍼파라미터를 찾으면 전체 훈련 세트로 모델을 다시 훈련함.
# 그리드 서치는 25개의 모델 중 가장 검증 점수가 높은 모델의 매개변수 조합으로 전체 훈련 세트를
# 자동으로 다시 훈련해줌.

# 이 객체는 gs 객체의 best_estimator_ 속성에 저장되어있음

dt = gs.best_estimator_
print(dt.score(train_input, train_target)) # 최적의 모델

In [ ]:
# 그리드 서치로 찾은 최적의 매개변수
print(gs.best_params_)

In [ ]:
# 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_result_ 속성의 mean_test_score키에 저장
gs.cv_results_

In [ ]:
print(gs.cv_results_['mean_test_score'])

In [ ]:
# 첫 번째 값이 가장 큼.
# 수동보단 numpy argmax() 함수를 사용하면 가장 큰 인덱스를 꺼낼 수 있음.
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

In [ ]:
# 과정 정리

# 1. 탐색할 매개변수 지정
# 2. 훈련 세트에서 그리드 서치 수행 > 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾기.
#  >> 이 조합은 그리드 서치 객체에 저장됨
# 3. 그리드 서치는 최상의 매개변수에서 (교차 검증에 사용한 훈련 세트가 아니라)
#   전체 훈련 세트를 사용해 최종 모델 훈련 >> 그리트 서치 객체에 저장

In [ ]:
# 결정 트리에서 min_impurity_decrease 는 노드를 분할하기 위한 불순도 감소 최소량을 지정
# max_depth 트리 깊이 제한.
# min_samples_split 노드를 나누기 위한 최소 샘플 수

params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
            'max_depth': range(5,20,1),
            'min_samples_split': range(2,100,10)}

# 넘파이 np.arange 배열 만들고 파이썬 range도 비슷한데 정수만 가능

# 총 9 x 15 x 10 = 1350 개의 교차 검증 횟수,
# 기본 5-폴드 교차검증 >> 총 6750개의 모델이 만들어짐.

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)
gs.fit(train_input, train_target)

In [ ]:
# 최상의 매개변수 조합
print(gs.best_params_)

In [ ]:
# 최상의 교차 검증 점수
print(np.max(gs.cv_results_['mean_test_score']))

랜덤 서치(Random Search)

- 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체 전달.

In [ ]:
# scipy에서 확률 분포 클래스 임포트
from scipy.stats import uniform, randint

# uniform과 randint클래스는 주어진 범위에서 고르게 값을 뽑음. == 균등분포에서 샘플링한다.
# randint == 정수값, uniform 실수값.

In [ ]:
rgen = randint(0,10)
rgen.rvs(10)

In [ ]:
np.unique(rgen.rvs(1000), return_counts=True)

In [ ]:
ugen = uniform(0,1)
ugen.rvs(10)

In [ ]:
np.unique(ugen.rvs(1000),return_counts=True)

In [ ]:
# 탐색할 매개변수 딕셔너리
# min_samples_leaf 매개변수 추가 == 리프 노드가 되기 위한 최소 샘플 개수
# 어떤 노드가 분할하여 만들어질 자식 노드 샘플 수가 이 값보다 작을 경우 분할 x

params = {'min_impurity_decrease':uniform(0.0001,0.001),
            'max_depth': randint(20,50),
            'min_samples_split': randint(2,25),
            'min_samples_leaf':randint(1,25)}

In [ ]:
# 샘플링 횟수는 사이킷런의 랜덤 서치 클래스인 RandomoizedSearchCV의 n_iter 매개변수에 지정

from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100,
                                                    n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

In [ ]:
print(gs.best_params_)

In [ ]:
# 교차 검증 점수
print(np.max(gs.cv_results_['mean_test_score']))

In [ ]:
# 최적의 모델 
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

코드 작성해 보기

In [ ]:
# 검증 세트 만들기

import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
wine.columns

In [ ]:
# 교차 검증 / 그리드 서치

In [ ]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split
train_input,test_input, train_target, test_target = train_test_split(data, target, test_size=0.2)


In [ ]:
from scipy.stats import uniform, randint

params = {'min_impurity_decrease': uniform(0.0001,0.001),
            'max_depth': randint(20,30),
            'min_samples_split': randint(2,25),
            'min_samples_leaf': randint(1,25)}
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(), params, n_iter=100, n_jobs=-1)
gs.fit(train_input, train_target)

In [ ]:
gs.best_params_

In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

In [ ]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

In [ ]:
print(dt.score(test_input, test_target))

In [ ]:
# ** 키워드 핵심 포인트 **

# ** 검증 세트 : 하이퍼파라미터 튜닝을 위해 모델을 평가할 떄,
#   테스트 세트를 사용하지 않기 위해 훈련 세트에서 다시 떼어 낸 데이터 세트

# ** 교차 검증 : 훈련 세트를 여러 폴드로 나눈 다음 한 폴드가 검증 세트의 역할을 하고
#   폴드에서는 모델을 훈련함.
#   모든 폴드에 대한 검증 점수를 얻어 평균하는 방법

# ** 그리드 서치 : 하이퍼파라미터 탐색을 자동화해주는 도구
#   탐색할 매개변수를 나열하면 교차 검증을 수행하여 가장 좋은 검증 점수의 매개변수 조합 선택
#   >> 마지막으로 이 매개변수 조합으로 최종 모델 훈련

# ** 랜덤 서치 : 연속된 매개변수 값을 탐색할 때 유용.
#   탐색할 값을 직접 나열하는 것이 아니고 탐색 값을 샘플링 할 수 있는 확률 분포 객체 전달.
#   지정된 횟수만큼 샘플링하여 교차 검증을 수행 >> 시스템 자원이 허락하는 만큼 탐색량 조절 가능

In [ ]:
# ** 패키지 함수**

# ** scikit-learn
#   ** cross_validate() 교차검증을 수행하는 함수
#   * scoring 매개변수에 검증을 사용할 평가 지표를 지정할 수 있음.
#   * 기본적으로 분류 모델은 정확도를 의미하는 'accuracy'. 회귀모델은 결정계수를 의미하는 'R2'
#   * cv 매개변수에 교차 검증 폴드 수나 스필리터 객체 지정 가능. 
#     기본값은 == 5
#     회기일 때는 KFold 클래스를 사용하고 분류일때는 StratifiedKFold 클래스 사용
#   * n_jobs=  교차 검증을 수행할 cpu 코어 수 지정. 기본값 == 1, -1지정 > 시스템 모든 코어 사용
#   * return_train_score = True, 훈련 세트의 점수도 반환, 기본값 ==False

#   ** GridSearchCV 교차검증으로 하이퍼파라미터 탐색 수행.
#     최상의 모델을 찾은 후 훈련 세트 전체를 사용해 최종 모델 훈련
#     * 첫 번째 매개변수로 그리드 서치를 수행할 모델 객체 전달.
#     * 두 번째 매개변수에는 탐색할 모델의 매개변수와 값 전달.
#     * scoring, cv, n_jobs, return_train_score 매개변수 cross_validate() 함수와 동일

#   ** RandomizedSearchCV 교차 검증으로 랜덤한 하이퍼파라미터 탐색 수행.
#     최상의 모델 찾은 후 훈련세트 전체를 사용해 최종 모델 훈련
#     * 첫 번째 매개변수는 그리드 서치로 수행할 모델 객체전달
#     * 두 번째는 매개변수에는 탐색할 모델과 매개변수와 확률 분포 객체 전달
#     * scoring, cv, n_jobs, return_train_score 매개변수는 cross_validate() 함수와 동일

확인 문제

In [ ]:
1. 훈련 세트를 여러 개의 폴드로 나누고 폴드 1갠느 평가 용도,나머지는 훈련 용도
그 다음 모든 폴드가 평가 용도로 사용하게끔 폴드 개수만큼 이 과정을 반복하는 것은?

교차 검증

In [ ]:
2. 교차 검증을 수행하지 않는 함수나 클래스

1.cross_validate()
2.GridSearchCV
3.RandomizedSearchCV
4.train_test_split >>>>>>

In [ ]:
3. 마지막 RandomizedSearchCV 예제에서 DecisionTreeClassifier 클래스에  Spliter= 'random'
매개변수를 추가하고 다시 훈련해보기

In [56]:
# 검증 세트 만들기

import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
wine.columns

Index(['alcohol', 'sugar', 'pH', 'class'], dtype='object')

In [62]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine[['class']].to_numpy()

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2)

# 랜덤 교차 검증하기 DecisionTree 모델이용하여

from scipy.stats import uniform, randint
params = {'min_impurity_decrease': uniform(0.0001,0.001),
            'max_depth': randint(2,25),
            'min_samples_split': randint(1,20),
            'min_samples_leaf':randint(1,20)
            }

from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(DecisionTreeClassifier(), params, n_iter=100, n_jobs=-1)
rs.fit(train_input, train_target)
어떻게 넣으라는 거지....

SyntaxError: invalid syntax (1038704033.py, line 17)